# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [11]:
!cat publications-auto.tsv

cat: publications-auto.tsv: No such file or directory


## Import pandas

We are using the very handy pandas library for dataframes.

In [12]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [17]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0, parse_dates=['pub_date'])
publications


/var/folders/2y/h464qnw52wq0lwvhs1s03dy40000gp/T/ipykernel_4596/2711942550.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  publications = pd.read_csv("publications.tsv", sep="\t", header=0, parse_dates=['pub_date'])


,pub_date,title,venue,excerpt,citation,url_slug,paper_url,slides_url
0,2019-06-15,Glacier-Surface Velocity Derived Ice Volume an...,Frontiers in Earth Science,NaN,"Sattar, A., Goswami, A., Kulkarni, A. V., & Da...",glacier-surface-velocity,https://doi.org/10.3389/feart.2019.00105,NaN
1,2022-05-15,Reservoir Assessment Tool 2.0: Stakeholder dri...,Environmental Modelling & Software,NaN,"Das, P., Hossain, F., Khan, S., Biswas, N. K.,...",reservoir-assessment-tool-2,https://doi.org/10.1016/j.envsoft.2022.105533,NaN
2,2022-12-15,Building User-Readiness for Satellite Earth Ob...,AGU Advances,NaN,"Hossain, F., Das, P., Srinivasan, M., Tsontos,...",swot-mission-user-readiness,https://doi.org/10.1029/2022AV000680,NaN
3,2023-03-15,Understanding Volume Estimation Uncertainty of...,IEEE Journal of Selected Topics in Applied Ear...,NaN,"Khan, S., Hossain, F., Pavelsky, T., Parkins, ...",lakes-wetlands-volume-estimation,https://doi.org/10.1109/JSTARS.2023.3250354,NaN
4,2023-12-15,Reservoir Assessment Tool Version 3.0: A Scala...,Geoscientific Model Development Discussions,NaN,"Minocha, S., Hossain, F., Das, P., Suresh, S.,...",reservoir-assessment-tool-3,https://doi.org/10.5194/gmd-2023-130,NaN
5,2024-01-15,Satellite-based Tracking of Reservoir Operatio...,Remote Sensing of Environment,NaN,"Suresh, S., Hossain, F., Minocha, S., Das, P.,...",satellite-reservoir-flood-management,https://doi.org/10.1016/j.rse.2024.114149,NaN
6,2024-01-15,ResORR: A globally scalable and satellite data...,Environmental Modelling & Software,NaN,"Das, P., Hossain, F., Minocha, S., Suresh, S.,...",resorr-global-river-flow,https://doi.org/10.1016/j.envsoft.2024.106026,NaN
7,2024-01-15,Assessing Snow Cover Patterns in the Indus-Gan...,Environmental Challenges,NaN,"Dixit, A., Goswami, A., Jain, S., & Das, P. (2...",snow-cover-hindu-kush-himalayas,https://doi.org/10.1016/j.envc.2023.100834,NaN
8,2024-03-15,Reservoir Assessment Tool (RAT): A Python Pack...,Digital Water,NaN,"S. Minocha, Das, P., and F. Hossain. Reservoir...",reservoir-assessment-tool-package,NaN,NaN
9,2024-04-15,Remote Sensing of Snow Cover Dynamics and Clim...,Climate Dynamics,NaN,"Dixit, A., Goswami, A., Jain, S. K., & Das, P....",snow-cover-river-basins,https://doi.org/10.1007/s00382-024-07280-5,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [18]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [19]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date.year) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date.year) + "-" + item.url_slug
    year = item.pub_date.year
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""

    md += """\ncategory: """ + item.category
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date.date()) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.slides_url)) > 5:
        md += "\nslidesurl: '" + item.slides_url + "'"

    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"

    if len(str(item.slides_url)) > 5:
        md += "\n[Download slides here](" + item.slides_url + ")\n" 

    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [16]:
!ls ../_publications/

2019-glacier-surface-velocity.md
2022-reservoir-assessment-tool-2.md
2022-swot-mission-user-readiness.md
2023-lakes-wetlands-volume-estimation.md
2023-reservoir-assessment-tool-3.md
2024-forecasting-informed-reservoir-operations.md
2024-reservoir-assessment-tool-package.md
2024-resorr-global-river-flow.md
2024-satellite-reservoir-flood-management.md
2024-snow-cover-hindu-kush-himalayas.md
2024-snow-cover-river-basins.md
2024-swot-storage.md
2024-thorr.md


In [16]:
!cat ../_publications/2009-10-01-paper-title-number-1.md

---
title: "Paper Title Number 1"
collection: publications
permalink: /publication/2009-10-01-paper-title-number-1
excerpt: 'This paper is about the number 1. The number 2 is left for future work.'
date: 2009-10-01
venue: 'Journal 1'
slidesurl: 'http://academicpages.github.io/files/slides1.pdf'
paperurl: 'http://academicpages.github.io/files/paper1.pdf'
citation: 'Your Name, You. (2009). &quot;Paper Title Number 1.&quot; <i>Journal 1</i>. 1(1).'
---
This paper is about the number 1. The number 2 is left for future work.

[Download slides here](http://academicpages.github.io/files/slides1.pdf)

[Download paper here](http://academicpages.github.io/files/paper1.pdf)

Recommended citation: Your Name, You. (2009). "Paper Title Number 1." <i>Journal 1</i>. 1(1).

# Do the opposite: generate a TSV from a list of markdown files

In [10]:
import os
import yaml
import pandas as pd

# Define the directory containing the publication files
publications_dir = '/Users/pdas47/cv/_publications'

# Define the output TSV file
output_tsv = '/Users/pdas47/cv/markdown_generator/publications-auto.tsv'

# Define the header for the TSV file
header = ['pub_date', 'title', 'venue', 'excerpt', 'citation', 'url_slug', 'paper_url', 'slides_url']

# Initialize a list to store the publication data
publications_data = []

# Function to extract metadata from the markdown file
def extract_metadata(filepath):
    with open(filepath, 'r') as file:
        content = file.read()
        try:
            front_matter = content.split('---')[1]
            metadata = yaml.safe_load(front_matter)
            return metadata
        except yaml.YAMLError as e:
            print(f"Error parsing YAML in file {filepath}: {e}")
            return {}

# Iterate over the files in the publications directory
for filename in os.listdir(publications_dir):
    if filename.endswith('.md'):
        filepath = os.path.join(publications_dir, filename)
        metadata = extract_metadata(filepath)
        pub_date = metadata.get('date', '')
        title = metadata.get('title', '')
        venue = metadata.get('venue', '')
        excerpt = metadata.get('excerpt', '')
        citation = metadata.get('citation', '')
        url_slug = metadata.get('permalink', '').split('/')[-1]
        paper_url = metadata.get('paperurl', '')
        slides_url = metadata.get('slidesurl', '')
        publications_data.append([pub_date, title, venue, excerpt, citation, url_slug, paper_url, slides_url])

# Create a DataFrame and write to TSV
df = pd.DataFrame(publications_data, columns=header)
df.to_csv(output_tsv, sep='\t', index=False)

print(f'TSV file created: {output_tsv}')

Error parsing YAML in file /Users/pdas47/cv/_publications/2024-damnet.md: while parsing a block mapping
  in "<unicode string>", line 2, column 1:
    title: "Damnet: A visualization  ... 
    ^
expected <block end>, but found '<scalar>'
  in "<unicode string>", line 11, column 112:
     ... s as transit hubs for the world's water. Journal of Visualizatio ... 
                                         ^
Error parsing YAML in file /Users/pdas47/cv/_publications/2024-thorr.md: while parsing a block mapping
  in "<unicode string>", line 2, column 1:
    title: "Reconstruction of the Hy ... 
    ^
expected <block end>, but found '<scalar>'
  in "<unicode string>", line 8, column 15:
    venue: 'Earth's Future'
                  ^
TSV file created: /Users/pdas47/cv/markdown_generator/publications-auto.tsv
